In [1]:
# Import packages.
import cvxpy as cp
import numpy as np
import os


def sigmoid_Activation_Class_Predictor(x):
    class_probablity = np.empty(x.shape[0])
    class_probablity = class_probablity.astype('float32')
    class_probablity = 1 / (1 + np.exp(-x))
    #print("class_probablity is",class_probablity)
    class_probablity= np.where(class_probablity < 0.5,-1, class_probablity) 
    #print(class_probablity)
    class_probablity= np.where(class_probablity > 0.5, 1, class_probablity)
    #print(class_probablity)
    return class_probablity
  


def accuracy_percentage(actual_value: np.ndarray, predicted_value: np.ndarray):
    #print(actual_value.shape,"  ",predicted_value.shape)
    #count which outcomes are same
    matches = (actual_value == predicted_value).sum()
    #print("matches", matches)
    total_samples = predicted_value.shape[0]
    #print("total_samples", total_samples)
    percentage_accuracy = (matches/total_samples)*100
    return percentage_accuracy

def least_Squares_Model_Prediction(test_input_data: np.ndarray,weights:np.ndarray, beta:np.ndarray):
    return test_input_data@weights + beta

def logistic_Loss_Model_Prediction(test_input_data: np.ndarray,weights:np.ndarray, beta:np.ndarray):
    return test_input_data@weights + beta

def hinge_Loss_Model_Prediction(test_input_data: np.ndarray,weights:np.ndarray, beta:np.ndarray):
    return test_input_data@weights + beta


path = os.path.split('/users/vikas/Documents/ionosphere/ionosphere.csv')[0]
print('check if path data file path is correct and exists : ', os.path.exists(path))


#import feature vectors and actual output values, form phi x_train, y_train vector set, actual output is mapped as g=1,b=-1
import pandas as pd
df=pd.read_csv('/users/vikas/Documents/ionosphere/ionosphere.csv', sep=',',header= 0)
df = df.loc[:, df.columns != '35']
df = df.astype(float)
data=df.values.tolist()
raw_Csv_Data=np.array(data)
x_train= np.array(raw_Csv_Data)
x_train=x_train[0:300,0:34]
#print("x_train data is :",x_train)
df2=pd.read_csv('/users/vikas/Documents/ionosphere/ionosphere.csv', sep=',',header=0)
df2 = df2.astype(str)
data2=df2.values.tolist()
raw_Csv_Data2=np.array(data2)
y_train= np.array(raw_Csv_Data2)
y_train= np.where(y_train=='g', 1, y_train) 
y_train= np.where(y_train=='b', -1, y_train)
y_train=y_train[0:300,34:]
y_train = y_train.astype(np.float32)
#print("y_train data is :",y_train)
#print("size of x_train,y_train",x_train.shape,y_train.shape)

x_test= np.array(raw_Csv_Data)
x_test=x_test[300:,0:34]

y_test= np.array(raw_Csv_Data2)
y_test= np.where(y_test=='g', 1, y_test) 
y_test= np.where(y_test=='b', -1, y_test)
y_test=y_test[300:,34:]
y_test = y_test.astype(np.float32)
#print("size of x_test,y_test",x_test.shape,y_test.shape)
#print("x_test data is :",x_test)
#print("y_test data is :",y_test)




# Defining and solving the least squares problem .
# basis function out of total 34 features , we will use 17 features,  hence 17 basis function which are quadratic are implemented
# the reason behind choosing quadratic function as basis function is that it gives good accuracy without overfitting the model
# which ususally happens with higher order power polynomials
# forming the phi matrix by applying basis function to the input vectors
phi_train_raw=x_train[0:300,0:34]
#print(phi_train_raw)

phi_train=np.empty((1,34), dtype=float)
nrows = phi_train_raw.shape
for i in range(nrows[0]):
    row = phi_train_raw[i, :]  
    #row=[1,2,3]
    # print(row)
    a=float(1)
    #print("a is : ",a)
    b=np.sum((np.square(row))*2,dtype = np.float32)
    #print("b is : ",b)
    c=np.sum((np.square(row))*2,dtype = np.float32)
    #print("c is : ",c)
    d=np.sum((np.square(row))*4,dtype= np.float32)
    #print("d is : ",d)
    e=np.sum((np.square(row))*5,dtype = np.float32)
    f=np.sum((np.square(row))*6,dtype = np.float32)
    h=np.sum((np.square(row))*7,dtype = np.float32)
    i=np.sum((np.square(row))*8,dtype = np.float32)
    j=np.sum((np.square(row))*9,dtype= np.float32)
    k=np.sum((np.square(row))*10,dtype= np.float32)
    newrow = [1,b,0,c,0,2,d,0,e,0,f,0,0,0,h,0,i,0,j,0,k,0,b,0,b,0,b,0,b,0,b,0,b,0]
    phi_train = np.vstack([phi_train, newrow])
    
#Phi_train matrix contains all basis function outputs for all 300 feature vectors
phi_train = phi_train[1:,0:]
#print("phi_train.shape is = " , phi_train.shape)
#calculation of pseudo inverse of matrox phi_train
pseudo_inverse= np.linalg.pinv(phi_train)
#calculation of theta - parameter values by multiplying it with pseudo inverse
theta_vector=np.matmul(pseudo_inverse,y_train)
#print("theta_vector  is = ",theta_vector)


#obtaining phi matrix for test data feature vectors total = 51
phi_test_raw=x_test[0:,0:34]
#print("phi_test_raw shape is + ",phi_test_raw.shape)
phi_test=np.empty((1,34), dtype=float)
nrows = phi_test_raw.shape
for i in range(nrows[0]):
    row = phi_test_raw[i, :]  
    #row=[1,2,3]
    # print(row)
    a=float(1)
    #print("a is : ",a)
    b=np.sum((np.square(row))*2,dtype = np.float32)
    #print("b is : ",b)
    c=np.sum((np.square(row))*2,dtype = np.float32)
    #print("c is : ",c)
    d=np.sum((np.square(row))*4,dtype= np.float32)
    #print("d is : ",d)
    e=np.sum((np.square(row))*5,dtype = np.float32)
    f=np.sum((np.square(row))*6,dtype = np.float32)
    h=np.sum((np.square(row))*7,dtype = np.float32)
    i=np.sum((np.square(row))*8,dtype = np.float32)
    j=np.sum((np.square(row))*9,dtype= np.float32)
    k=np.sum((np.square(row))*10,dtype= np.float32)
    newrow = [1,b,0,c,0,2,d,0,e,0,f,0,0,0,h,0,i,0,j,0,k,0,b,0,b,0,b,0,b,0,b,0,b,0]
    phi_test = np.vstack([phi_test, newrow])
    

phi_test = phi_test[1:,0:]
#print("phi_test shape is = ",phi_test.shape)
#predicting outcomes for test data
predicted_results= np.matmul(phi_test,theta_vector)
print("predicted_results for test data using least squares loss using phi matrix is = ",predicted_results)
#prediction of class using outcomes using sigmoid function
predicted_class_output1=sigmoid_Activation_Class_Predictor(predicted_results)
print("predicted_class_output for test data using least squares problem solved using phi matrix  is  = ",predicted_class_output1)
Percentage_accuracy1 =  accuracy_percentage(y_test,predicted_class_output1)            
print("Percentage_accuracy for test data using least squares loss model using phi matrix is  = ",Percentage_accuracy1)



w,b = cp.Variable((34,1)),cp.Variable(1)
x,y = cp.Parameter(), cp.Parameter()
x=x_train
y=y_train

# Defining and solving the least squares problem using CVX package .
loss = cp.sum_squares( x@w + b - y)
prob = cp.Problem(cp.Minimize(loss))
prob.solve()
#print("The optimal w for least squares using CVX is = ",w.value)
#print("The optimal b for least squares using CVX is = ",b.value)
predicted_value1=least_Squares_Model_Prediction(x_test,w.value,b.value)
print("predicted_value for test data using least squares by CVX is  = ",predicted_value1)
predicted_class_output1=sigmoid_Activation_Class_Predictor(predicted_value1)
print("predicted_class_output of least squares by CVX is  = ",predicted_class_output1)
Percentage_accuracy1 =  accuracy_percentage(y_test,predicted_class_output1)            
print("Percentage_accuracy for test data using least squares loss model by CVX is  = ",Percentage_accuracy1)

w,b = cp.Variable((34,1)),cp.Variable(1)
x,y = cp.Parameter(), cp.Parameter()
x=x_train
y=y_train.transpose()

# Defining and solving the logistic loss problem .
loss=cp.sum(cp.logistic(-1*(y@(x@w + b))))
prob1=cp.Problem(cp.Minimize(loss))
prob1.solve()
#print("The optimal w for logistic loss problem is = ",w.value)
#print("The optimal b for logistic loss problem is = ",b.value)
predicted_value2=logistic_Loss_Model_Prediction(x_test,w.value,b.value)
print("predicted_value2 for test data using logistic_Loss_Model is  = ",predicted_value2)
predicted_class_output2=sigmoid_Activation_Class_Predictor(predicted_value2)
print("predicted_class_output for test data using logistic_Loss_Model is  = ",predicted_class_output2)
Percentage_accuracy2 =  accuracy_percentage(y_test,predicted_class_output2)            
print("Percentage_accuracy for test data using logistic_Loss_Model is  = ",Percentage_accuracy2)



w,b = cp.Variable((34,1)),cp.Variable(1)
x,y = cp.Parameter(), cp.Parameter()
x=x_train
y=y_train.transpose()


# Defining and solving the Hinge Loss problem .
cost3=(cp.maximum(0,(1-(y@(x@w + b)))))
prob3 = cp.Problem(cp.Minimize(cp.sum(cost3)))
prob3.solve(solver=cp.ECOS)
#print("The optimal for the Hinge Loss problem w is = ",w.value)
#print("The optimal for the Hinge Loss problem b is = ",b.value)
predicted_value3=hinge_Loss_Model_Prediction(x_test,w.value,b.value)
print("predicted_value for test data using hinge_Loss_Model is  = ",predicted_value3)
predicted_class_output3=sigmoid_Activation_Class_Predictor(predicted_value3)
print("predicted_class_output for test data using hinge_Loss_Model is  = ",predicted_class_output3)
Percentage_accuracy3 =  accuracy_percentage(y_test,predicted_class_output3)            
print("Percentage_accuracy for test data using hinge_Loss_Model is  = ",Percentage_accuracy3)



check if path data file path is correct and exists :  True
predicted_results for test data using least squares loss using phi matrix is =  [[ 0.43633888]
 [ 0.19936219]
 [-0.16571148]
 [ 0.13375372]
 [ 0.22007272]
 [ 0.3473249 ]
 [ 0.42545878]
 [ 0.33553541]
 [ 0.20767185]
 [ 0.32043878]
 [ 0.00692927]
 [-0.10670981]
 [-0.19330854]
 [ 0.16077312]
 [ 0.34600128]
 [ 0.12602189]
 [ 0.46368679]
 [ 0.14549278]
 [ 0.25248213]
 [ 0.49528434]
 [-0.30118114]
 [ 0.18099664]
 [ 0.02952117]
 [ 0.55785416]
 [ 0.32615255]
 [ 0.15933092]
 [ 0.14560029]
 [ 0.24303849]
 [ 0.40296475]
 [ 0.47756239]
 [ 0.28513134]
 [ 0.35390463]
 [ 0.38805138]
 [ 0.01814209]
 [ 0.01614143]
 [ 0.05271426]
 [ 0.28497736]
 [ 0.07264799]
 [ 0.24076741]
 [ 0.2035166 ]
 [ 0.42122834]
 [ 0.18735596]
 [ 0.12864747]
 [ 0.33045737]
 [ 0.21747127]
 [ 0.30706711]
 [ 0.13083771]
 [ 0.01102554]
 [ 0.21997043]
 [-0.01771663]
 [ 0.13775123]]
predicted_class_output for test data using least squares problem solved using phi matrix  is  =